# LangChain Tool Calling

For more information: https://langchain-ai.github.io/langgraph/how-tos/tool-calling/

In [9]:
%pip install pydantic python-dotenv langchain langchain-openai langchain-community

Note: you may need to restart the kernel to use updated packages.


In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
from langchain_core.tools import tool

@tool
def get_weather(location: str) -> str:
    """
    Generate a weather message for a given location.

    Args:
        location (str): The location for which to generate the weather report.

    Returns:
        str: A weather message for the specified location.
    """
    return f"The weather in {location} is sunny with a chance of clouds."

In [12]:
# Tools conform to the Runnable interface, which means you can run a tool using the invoke method

get_weather.invoke("Tandil")

'The weather in Tandil is sunny with a chance of clouds.'

In [13]:
tool_call = {
    "type": "tool_call",
    "id": "1",
    "args": { "location": "Mar del Plata" }
}

get_weather.invoke(tool_call) # returns a ToolMessage object

ToolMessage(content='The weather in Mar del Plata is sunny with a chance of clouds.', name='get_weather', tool_call_id='1')

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

SYSTEM_PROMPT = """
You report the weather of a given location using emojis. 

Use the following format for your answers:
1. Tandil: ☀️ - 🌡
2. London: ️🌧️ - ❄️
"""

model = init_chat_model(model="gpt-4.1-nano", temperature=0)

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content="What's the weather like in Paris today?"),
    ]
)

# Bind schema to model
model_with_tools = model.bind_tools([get_weather])

# Invoke the model to produce structured output that matches the schema
response = model_with_tools.invoke(chat_template.format_messages())


In [20]:
print(response.tool_calls)

[{'name': 'get_weather', 'args': {'location': 'Paris'}, 'id': 'call_C2IktCAiqcg9R2N9vErx6h3a', 'type': 'tool_call'}]


In [21]:
tool_call = response.tool_calls[0]

tool_return = get_weather.invoke(tool_call)

In [22]:
tool_return

ToolMessage(content='The weather in Paris is sunny with a chance of clouds.', name='get_weather', tool_call_id='call_C2IktCAiqcg9R2N9vErx6h3a')

In [25]:
msgs = chat_template.format_messages() + [response] + [tool_return]

msgs

[SystemMessage(content='\nYou report the weather of a given location using emojis. \n\nUse the following format for your answers:\n1. Tandil: ☀️ - 🌡\n2. London: ️🌧️ - ❄️\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the weather like in Paris today?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_C2IktCAiqcg9R2N9vErx6h3a', 'function': {'arguments': '{"location":"Paris"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 129, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-C5xrbw6IES5wxaBRbiXAKCfVVyL4q', 'service_tier': 'defaul

In [27]:
model_with_tools.invoke(msgs).content

'Paris: ☀️ - ☁️'